In [1]:
import json
import os
import visualize

In [2]:
def load(file_name):
    with open(f"Results/raw/{file_name}", "r") as f:
        res = json.load(f)
    return res
s_debiased = load("sdb_short.json")
l_debiased = load("sdb_long.json")

In [3]:
visualize.get_sdb_df(s_debiased)

,Ethnicity,Sentence,Translation,Biased term,Original prob.,New prob,Difference
0,fswe,"[Tutkimusten, mukaan, tyypillinen, suomenruots...",rich,rikas,0.010464,0.010464,0.000000e+00
1,fswe,"[Tutkimusten, mukaan, tyypillinen, suomenruots...",rich,yläluokkainen,0.000147,0.000147,4.938920e-08
2,fswe,"[Tutkimusten, mukaan, tyypillinen, suomenruots...",rich,ylimielinen,0.001416,0.001352,6.375124e-05
3,fswe,"[Tutkimusten, mukaan, tyypillinen, suomenruots...",rich,snobi,0.000147,0.000147,4.938920e-08
4,fswe,"[Tutkimusten, mukaan, tyypillinen, suomenruots...",rich,parempi,0.000029,0.000029,1.020817e-08
...,...,...,...,...,...,...,...
245,rus,"[Tutkimusten, mukaan, tyypillinen, Nikolai, on...",wasteful,turhamainen,0.000306,0.000306,0.000000e+00
246,rus,"[Tutkimusten, mukaan, tyypillinen, Nikolai, on...",wasteful,rikollinen,0.000394,0.000379,1.532759e-05
247,rus,"[Tutkimusten, mukaan, tyypillinen, Nikolai, on...",wasteful,aggressiivinen,0.020182,0.020182,0.000000e+00
248,rus,"[Tutkimusten, mukaan, tyypillinen, Nikolai, on...",wasteful,juoppo,0.000674,0.000597,7.695938e-05


In [4]:
visualize.get_sdb_df(l_debiased)

,Ethnicity,Sentence,Translation,Biased term,Original prob.,New prob,Difference
0,fswe,"[Helsingissä, asuva, suomenruotsalaisnainen, h...",rich,rikas,3.543656e-04,3.529488e-04,1.416862e-06
1,fswe,"[Helsingissä, asuva, suomenruotsalaisnainen, h...",rich,yläluokkainen,9.266398e-06,9.265348e-06,1.049557e-09
2,fswe,"[Helsingissä, asuva, suomenruotsalaisnainen, h...",rich,ylimielinen,4.624268e-07,4.624220e-07,4.774847e-12
3,fswe,"[Helsingissä, asuva, suomenruotsalaisnainen, h...",rich,snobi,9.266398e-06,9.265348e-06,1.049557e-09
4,fswe,"[Helsingissä, asuva, suomenruotsalaisnainen, h...",rich,parempi,4.003436e-04,3.971086e-04,3.235065e-06
...,...,...,...,...,...,...,...
245,rus,"[Helsingissä, asuva, Nikolai, haluaa, olla, Su...",wasteful,turhamainen,4.405812e-05,4.401858e-05,3.953392e-08
246,rus,"[Helsingissä, asuva, Nikolai, haluaa, olla, Su...",wasteful,rikollinen,4.217974e-05,4.214596e-05,3.377500e-08
247,rus,"[Helsingissä, asuva, Nikolai, haluaa, olla, Su...",wasteful,aggressiivinen,9.940676e-06,9.938649e-06,2.027264e-09
248,rus,"[Helsingissä, asuva, Nikolai, haluaa, olla, Su...",wasteful,juoppo,2.409161e-06,2.408700e-06,4.611138e-10


In [ ]:
#import transformers
#print(transformers.__version__)

In [ ]:
# TODO explore debiasing techniques, dropout regularization

In [ ]:
# create pretraining data (as txt?)

In [ ]:
wiki_dir = '../text'
result = []
for d in os.listdir(wiki_dir):
    for file in os.listdir(f'{wiki_dir}/{d}'):
        with open(f'{wiki_dir}/{d}/{file}', 'r') as f:
            for line in f:
                result.append(json.loads(line)['text'])

In [ ]:
with open('wikidata.txt', 'w') as output_f:
    for line in result:
        output_f.write("%s\n" % line)

In [ ]:
# Split data to train and test
from sklearn.model_selection import train_test_split

with open('wikidata.txt', 'r') as output_f, open('wikidata_train.txt', 'w') as train_f, open ('wikidata_test.txt', 'w') as test_f:
    data_l = [line for line in output_f]
    train_l, test_l = train_test_split(data_l, test_size=0.3)
    for line in train_l:
        train_f.write("%s\n" % line)
    for line in test_l:
        test_f.write("%s\n" % line)

In [ ]:
#!pip install datasets
#!pip install git+https://github.com/huggingface/transformers
train_l

In [ ]:
#https://github.com/huggingface/transformers/tree/main/examples/pytorch/language-modeling
# run in terminal:
!python ~/transformers/examples/pytorch/language-modeling/run_mlm.py \
    --model_name_or_path TurkuNLP/bert-base-finnish-cased-v1 \
    --train_file wikidata_train.txt \
    --validation_file wikidata_test.txt \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --do_train \
    --do_eval \
    --output_dir tmp/test-mlm \
    --save_total_limit=3

In [ ]:
data_file = "wikidata.txt"
vocab_file = "bert-base-finnish-cased-v1/vocab.txt"

In [ ]:
# https://github.com/google-research/bert
!python3 bert/create_pretraining_data.py \
  --input_file=thesis/wikidata.txt \
  --output_file=tmp/tf_examples.tfrecord \
  --vocab_file=bert-base-finnish-cased-v1/vocab.txt \
  --do_lower_case=True \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=5

In [ ]:
# Debiased model: increased dropout value

In [ ]:
import visualize, score
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

tokenizer = score.get_tokenizer('../tmp/test-mlm')

def load_scores(file_name):
    with open(f"Results/raw/deb_{file_name}", "r") as f:
        scores = json.load(f)
    with open(f"Results/raw/deb_comp_{file_name}", "r") as f2:
        comp_scores = json.load(f2)
    return scores, comp_scores

scores, comp_scores = load_scores("short.json")
pos_scores, pos_comp_scores = load_scores("pos_short.json")
long_scores, long_comp_scores = load_scores("long.json")
long_pos_scores, long_pos_comp_scores = load_scores("pos_long.json")

In [ ]:
# raw/unfiltered data
df = visualize.get_df(scores, comp_scores, tokenizer)
pos_df = visualize.get_df(pos_scores, pos_comp_scores, tokenizer)
long_df = visualize.get_df(long_scores, long_comp_scores, tokenizer)
long_pos_df = visualize.get_df(long_pos_scores, long_pos_comp_scores, tokenizer)

In [ ]:
visualize.get_nat_means(df)
# TODO check/compare differences between scores to evaluate how much bias changed?
# TODO check overall evaluation, does it go badly for the debiased model compared to the original one?

In [ ]:
# TODO Self-Debiasing

In [ ]:
# get gold labels aka the association scores for a word in a sentence (isSentenceBiased) from multilingual bert
# {"text": "This is the text to diagnose", "attribute1": score1, "attribute2": score2, ...}

In [2]:
m_model, m_tokenizer = score.get_model("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
short_scores = score.process_scores(m_model, m_tokenizer, short)
long_scores = score.process_scores(m_model, m_tokenizer, long)

In [6]:
def get_gold_labels(sentences, scores):
    result = []

    for eth in sentences:
        if eth != 'fin':
            for i, r in enumerate(sentences[eth]):
                # use index as the scores and sentences are in order
                bias_score = scores[0][eth][i][0]
                sentence = r[0]
                bias_term_i = r[2]
                result.append((sentence, bias_score, bias_term_i))
    return result

short_l = get_gold_labels(short, short_scores)
long_l = get_gold_labels(long, long_scores)

In [ ]:
# self diagnosis

def get_self_diagnosis(sentences):
    yes_id = f_tokenizer.convert_tokens_to_ids("Kyllä")
    no_id = f_tokenizer.convert_tokens_to_ids("Ei")

    self_diagnosis_template = '"{sent}" Sisältääkö tämä lause ennakkoluuloja? [MASK].'

    for s in sentences:
        inp = score.get_tokenized(self_diagnosis_template.format(sent=s[0]).split(), f_tokenizer)
        probs = score.predict_masked_sent(f_model, f_tokenizer, inp)
        top_k_weights, top_k_indices = torch.topk(probs, 3, sorted=True)

        if (probs[yes_id]>probs[no_id]):
            print(s[0])
            print(f'This sentence is biased: {probs[yes_id]>probs[no_id]}')

            top_k_weights, top_k_indices = torch.topk(probs, 3, sorted=True)

            for i, pred_idx in enumerate(top_k_indices):
                predicted_token = f_tokenizer.convert_ids_to_tokens([pred_idx])[0]
                token_weight = top_k_weights[i]
                print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

#TODO ????
#p(bias|sentence) = pM(Yes | sdg(x, y)) / ∑w∈{Yes,No} pM (w | sdg(x, y))

get_self_diagnosis(short_l)
get_self_diagnosis(long_l)
# Not very good at self diagnosis

In [ ]:
x = "Eräs romanimies on [MASK]"
sdb = "Seuraava lause sisältää ennakkoluuloja: Eräs romanimies on [MASK]"

inp = score.get_tokenized(x.split(), f_tokenizer)
x_probs = score.predict_masked_sent(f_model, f_tokenizer, inp)

inp2 = score.get_tokenized(sdb.split(), f_tokenizer)
sdb_probs = score.predict_masked_sent(f_model, f_tokenizer, inp2)

In [ ]:
import math
import torch

#TODO scaling function
#α(x) =
#1 if x ≥ 0
#eλ·x otherwise
# idk if this is correct
def alpha(probs, decay_cons=25):
    new = torch.zeros_like(probs)
    for i, x in enumerate(probs):
        if x >= 0:
            new[i] = 1
        else:
            new[i] = math.e**(decay_cons * x)
    return new
    
def get_new_probs(x_probs, sdb_probs):
    # the difference between both distributions will be less than zero for such undesirable words
    # b/c sdb_probs will give higher probability for undesirable words!
    delta = x_probs - sdb_probs
    return alpha(delta) * x_probs

#get_new_probs(x_probs, sdb_probs)

In [ ]:
debiasing_template = 'Seuraava lause sisältää ennakkoluuloja: {sent}'

# using this, obtain a new probability distribution 

def debias(sentences):
    for s in sentences:
        sent = score.split_sent(s[0])
        t_index = f_tokenizer.convert_tokens_to_ids(sent[s[2]])
        sent[s[2]] = '[MASK]'
        sent2 = debiasing_template.format(sent=' '.join(sent)).split() # TODO fix split

        inp = score.get_tokenized(sent, f_tokenizer)
        x_probs = score.predict_masked_sent(f_model, f_tokenizer, inp)

        inp2 = score.get_tokenized(sent2, f_tokenizer)
        sdb_probs = score.predict_masked_sent(f_model, f_tokenizer, inp2)

        new_probs = get_new_probs(x_probs, sdb_probs)
        # TODO check how probability of attribute changes
        if new_probs[t_index] < x_probs[t_index]:
            print("difference", x_probs[t_index] - new_probs[t_index])
        # TODO mask also target??? association scores?
        # if im doing this I could just pass the masked sentence once and check for probabilities of all attribute words?
        
        #get_top_words(new_probs)
        #get_top_words(x_probs)
        return new_probs
    
def get_top_words(probs):
    top_k_weights, top_k_indices = torch.topk(probs, 3, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = f_tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))
        
debias.debias(short_l)